In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
#x,y,c,
poseName = 'Penche'
Coords = np.load("RootCoordinates/"+poseName+"X.npy")
print(Coords)
print(len(Coords))

[[0.632734 0.372073 0.890703 ... 0.318147 0.423723 0.564912]
 [0.649171 0.410222 0.969969 ... 0.334479 0.358332 0.563117]
 [0.655377 0.431736 0.945887 ... 0.342718 0.328286 0.537246]
 ...
 [0.62866  0.505482 0.839337 ... 0.442761 0.32284  0.458389]
 [0.628602 0.508102 0.809137 ... 0.430537 0.320101 0.446278]
 [0.62671  0.497314 0.911519 ... 0.444839 0.312016 0.483267]]
870


In [3]:
#Helper Methods:
import math
def calcDistance(x1,y1,x2,y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [4]:
#Get the Mean Skeleton's coordinates
PoseMeanX = np.load("Stats/"+poseName+"MeanX.npy")
print(PoseMeanX)
print(len(PoseMeanX))
PoseMeanY = np.load("Stats/"+poseName+"MeanY.npy")
print(PoseMeanY)
print(len(PoseMeanY))
MeanTuples = list(zip(PoseMeanX, PoseMeanY))
print(MeanTuples)

[0.63005629 0.58423142 0.57813234 0.53919554 0.50144006 0.59291618
 0.62008335 0.65282488 0.5        0.49855253 0.45453837 0.40351882
 0.50192433 0.49238908 0.49182182 0.62889447 0.01014245 0.61279663
 0.         0.50594318 0.47881126 0.47272003 0.34806243 0.3130966
 0.38819208]
25
[-0.47484516 -0.49122071 -0.46925463 -0.43853003 -0.40463798 -0.51477767
 -0.56152993 -0.60087824 -0.5        -0.48027024 -0.45141688 -0.40083242
 -0.52012724 -0.61044769 -0.68674702 -0.46469391 -0.00796137 -0.45791381
  0.         -0.68372222 -0.64880791 -0.68064527 -0.34306633 -0.32171092
 -0.3847614 ]
25
[(0.6300562873563219, -0.4748451609195405), (0.5842314172413794, -0.49122071264367817), (0.5781323390804597, -0.46925462988505734), (0.5391955390804601, -0.43853002873563235), (0.501440055172414, -0.4046379770114939), (0.5929161839080468, -0.5147776701149424), (0.6200833471264368, -0.5615299287356319), (0.6528248770114945, -0.6008782379310343), (0.5, -0.5), (0.4985525264367807, -0.48027023908046), (0.4545

In [5]:
POSE = "Penche"
allDistanceVectors = []
for sample in Coords:
    xCoords = sample[0::3]
    #print(xCoords)
    yPos = sample[1::3]
    yCoords = [-y for y in yPos] # need to make it negative to be consistent - the mean skeleton's y saved the negative coords.
    SampleTuples = list(zip(xCoords, yCoords))
    #print(SampleTuples)
    longZip = list(zip(MeanTuples, SampleTuples))
    #print(longZip)
    distanceVector = []
    for ((x1,y1),(x2,y2)) in longZip:
        d = calcDistance(x1,y1,x2,y2)
        #print(d)
        distanceVector.append(d)
    allDistanceVectors.append(distanceVector)

print(allDistanceVectors)
print(len(allDistanceVectors))
print(len(allDistanceVectors[0]))
np.save("DataForSVM/"+POSE+"DistanceVectorsPerSample.npy", allDistanceVectors)

[[0.10280703869421608, 0.07352737446046725, 0.0673135719093956, 0.05407734755930696, 0.04637959125036222, 0.07927446946084794, 0.10956008531807934, 0.1536659075827351, 0.0, 0.005172495617996779, 0.042784657485110025, 0.08252082327497258, 0.010458470568841494, 0.03964906648455448, 0.08502250445338477, 0.09828414537089669, 0.012893905187165407, 0.10232309061304243, 0.0, 0.11020195529182122, 0.14749145670218144, 0.10892172248365045, 0.09360211711183715, 0.08936222627347294, 0.08015185543356616], [0.06739083889285856, 0.04330792542110109, 0.043483373836116904, 0.04773172729618802, 0.04820392395310899, 0.04586220516228142, 0.06455561260538524, 0.08594677226461835, 0.0, 0.006742973418514686, 0.039102751851406846, 0.07343906918321147, 0.004448889402443817, 0.039537726257888775, 0.08605151310854825, 0.07105451262092975, 0.012893905187165407, 0.06870842610844083, 0.0, 0.11024246640745133, 0.14934254748695155, 0.10759699899473053, 0.043711220446487675, 0.010678443967414217, 0.05986324376283701],

In [6]:
#These features are mixing coordinates and distances: 100 features per sample: 75 + 25
PoseAugFeaturesX = []
for i in range(len(Coords)):
    newList = list(Coords[i])+list(allDistanceVectors[i])
    PoseAugFeaturesX.append(newList)
    
print(PoseAugFeaturesX[0])
print(len(PoseAugFeaturesX[0]))
np.save("DataForSVM/"+POSE+"AugMeanDistFeaturesX.npy", PoseAugFeaturesX)

[0.632734, 0.372073, 0.8907029999999999, 0.5938410000000001, 0.418324, 0.884855, 0.577503, 0.40194399999999997, 0.8267009999999999, 0.528465, 0.385528, 0.8349789999999999, 0.47940900000000003, 0.363825, 0.871378, 0.61023, 0.437417, 0.8335819999999999, 0.659292, 0.459226, 0.894651, 0.718494, 0.461951, 0.790829, 0.5, 0.5, 0.772072, 0.493835, 0.47814899999999994, 0.764382, 0.41205400000000003, 0.45647699999999997, 0.737544, 0.32425400000000004, 0.423784, 0.702176, 0.512233, 0.521891, 0.775718, 0.4959, 0.649941, 0.851112, 0.510098, 0.769782, 0.794647, 0.630681, 0.36642600000000003, 0.8919349999999999, 0.0, 0.0, 0.0, 0.612285, 0.35559199999999996, 0.8876489999999999, 0.0, 0.0, 0.0, 0.546949, 0.786011, 0.715932, 0.544788, 0.78072, 0.637707, 0.49996700000000005, 0.7861039999999999, 0.753919, 0.285538, 0.412723, 0.658291, 0.28750400000000004, 0.40732999999999997, 0.6567029999999999, 0.318147, 0.42372299999999996, 0.564912, 0.10280703869421608, 0.07352737446046725, 0.0673135719093956, 0.0540773